# Lecture on the Wigner-Dyson nearest-neighbour distribution

    by M.Süzen
    (c) 2025

In this lecture, we will generate the spectral density $ \rho(\lambda)$ emprically from a sample of matrices sampled from the Gaussian Orthogonal Ensemble (GOE) and compare the nearest neighbour spacing against the theory.

## Load Leymosun components

We assume that you have installed the package. We will import the following:

`goe` : Gaussian Orthogonal Matrix generator.   
`empirical_spectral_density`: Compute eigenvalues and their density $\rho(\lambda)$. Here we will only use spectra from this method.   
`unfold_spectra`, `pdf`,  `wigner_spacing` and `bootstrap_observed_matrix_ci`.


See each method's documentation for more details. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import leymosun
from leymosun.gaussian import goe, wigner_spacing
from leymosun.matrix import ensemble
from leymosun.spectral import empirical_spectral_density, unfold_spectra
from leymosun.stats import pdf, bootstrap_observed_matrix_ci


leymosun.__version__

## Nearest-neigbour spacing

The semicircle law traces its origins to Wigner's work on the calculations of the quantum mechanical Hamiltonian [Wigner55]. The resulting distribution of the eigenvalue spacinings $s_{i}$ follows the following asymptotic 
distribution. The nearest neighbor spacing distribution (NNSD) [Wigner57]: 
$$ P(s) = \frac{\pi}{2} \cdot s \exp(-\frac{\pi}{4} \cdot s^{2})$$


## Numerical Experiment

We conduct out numerical experiment using the tools provided by `leymosun` package. Emprical GOE ensemble of size 20 is generated for square matrices of 5000 by 5000. Smaller matrices may not capture the Wigner-Dyson distribution.

In [ ]:
matrix_order = 1000 
ensemble_size = 20
ensemble_sample = ensemble(matrix_order, ensemble_size, goe)

In [ ]:
eigenvalues, densities, locations = empirical_spectral_density(
    ensemble_sample, scale="wigner",  locations = np.arange(-2.05, 2.1, 0.05),

)

In [ ]:
eigenvalues.shape, densities.shape

In [ ]:
nn_spacing = []
locations = np.arange(0.0, 5.0, 0.1) # NNSD locations
nnsd_densities = []
for eigenvalue in eigenvalues:
    ueigenvalue,  ueigenvalues_nn, _ = unfold_spectra(eigenvalue, deg_max=30, iqr=True)
    density, _locations = pdf(ueigenvalues_nn, locations=locations)
    nnsd_densities.append(density)
swigner = wigner_spacing(_locations)

In [ ]:
observed_mean, observed_upper, observed_lower = bootstrap_observed_matrix_ci(np.array(nnsd_densities))

## Visualisation

We visalise the results in a single graph. A continous red line is the exact Wigner NN spacing from the theory, and bars are from our numerical experiments. 

In [ ]:
yerr = np.array([observed_mean-observed_lower, observed_upper-observed_mean])
plt.bar(_locations, observed_mean, width=0.02, alpha=0.3, align='center') 
plt.errorbar(_locations, observed_mean, yerr=yerr, fmt=' ', capsize=5)  
plt.plot(_locations, swigner, color='red')

plt.title("Wigner-Dyson  Neareast-Neigbour \n Distribution \n  Unfolding GOE with Leymosun")
plt.xlabel("Unfolded Spectrum Locations")
plt.ylabel("Neareast-neighbour Density")

## Conclusion

We observe good match for demonstration purposes here. 


## Reference

[Wigner57] Wigner, E. P. (1957). "Statistical Theory of Spectra: Fluctuations", Annals of Mathematics, 65(2), 203–216.

QED